In [ ]:
import os

import geopandas as gpd
import numpy as np
import pandas as pd
import rioxarray as rio
import xarray as xr

# Extract deposition for 2017 to 2021 for the 1000 Lakes

See e-mail from Øyvind Kaste received 14.10.2022.

In [ ]:
data_fold = r"/home/jovyan/shared/critical_loads/raster/deposition"

## 1. EMEP data

In [ ]:
# Read data from Max
emep_dict = {}
pars = ["NHy", "NOx", "SOx"]
for par in pars:
    csv_path = os.path.join(data_fold, "from_max_2022", f"SiteDep{par}.csv")
    df = pd.read_csv(csv_path, skiprows=2)
    emep_dict[par] = df
    print(f"{len(df)} stations for {par}.")

## 2. NILU data

In [ ]:
%%time

# met.no CRS used by NULI
met_crs = r"+proj=lcc +lat_0=63 +lon_0=15 +lat_1=63 +lat_2=63 +no_defs +R=6.371e+06"

# Example met.no dataset with same geo transfrom etc. (because this wasn't specified by NILU)
met_ds = xr.open_dataset(
    r"https://thredds.met.no/thredds/dodsC/metpplatest/met_forecast_1_0km_nordic_latest.nc"
)

# Read NILU 1km data
pars = ["nh4", "no3", "xso4"]
years = range(2017, 2022)
ds = xr.Dataset({})
for par in pars:
    # Create empty DataArray with correct dims
    da = xr.DataArray(
        dims=("y", "x", "year"),
        coords={"x": met_ds.x, "y": met_ds.y, "year": years},
    )
    da.rio.write_nodata(np.nan, inplace=True)

    # Add NILU data from numpy arrays
    for year in years:
        arr_path = os.path.join(
            data_fold,
            "from_nilu_2022",
            "nilu_dep_2017-21_met_1km_grid",
            f"tot_dep_{par}_{year}.npy",
        )
        data = np.load(arr_path)

        # Add values to DataArray
        da.loc[dict(year=year)] = np.flip(data, axis=0)

    # Add DataArray to DataSet
    ds[par] = da

# Update attrs with spatial info from met dataset
ds.rio.write_crs(met_crs, inplace=True,).rio.set_spatial_dims(
    x_dim="x",
    y_dim="y",
    inplace=True,
).rio.write_coordinate_system(inplace=True)
ds.rio.write_transform(met_ds.rio.transform())

# Wite example to GeoTIFF for testing in ArcGIS
# ds['nh4'].sel(year=2017).rio.to_raster("test.tif")

ds

In [ ]:
%%time

# Some stations lie just outside of met's grid. Interpolate using 'nearest neighbour'
# (otherwise xarray retrun NaN for ~60 points). Takes about 10 mins
ds2 = ds.transpose("year", "y", "x").rio.interpolate_na(method="nearest")
ds2

## 3. Extract values for points

In [ ]:
# Get station co-ords from Max's data
stn_df = emep_dict["NHy"][["StationCode", "Latitude", "Longitude"]]
stn_gdf = gpd.GeoDataFrame(
    stn_df,
    geometry=gpd.points_from_xy(
        stn_df["Longitude"], stn_df["Latitude"], crs="epsg:4326"
    ),
)

# Get x and y in grid co-ords
stn_gdf["x"] = stn_gdf.to_crs(met_crs)["geometry"].x
stn_gdf["y"] = stn_gdf.to_crs(met_crs)["geometry"].y
stn_gdf.head()

In [ ]:
# Extract values for points
x_indexer = xr.DataArray(
    stn_gdf["x"], dims=["StationCode"], coords=[stn_gdf["StationCode"]]
)
y_indexer = xr.DataArray(
    stn_gdf["y"], dims=["StationCode"], coords=[stn_gdf["StationCode"]]
)
pts_ds = ds2.sel(x=x_indexer, y=y_indexer, method="nearest")
pts_df = pts_ds.to_dataframe().reset_index()
display(pts_df.describe())

# Reformat to have similar structure to Max's data
pts_df = pts_df[["StationCode", "year"] + pars]
pts_df = pts_df.melt(id_vars=["StationCode", "year"])
pts_df.set_index(["StationCode", "year", "variable"], inplace=True)
pts_df = pts_df.unstack("year")
pts_df.columns = pts_df.columns.get_level_values(1)
pts_df.columns.name = ""
pts_df.reset_index(inplace=True)
pts_df.rename({"StationCode": "station_code"}, inplace=True, axis="columns")
pts_df = pts_df.round(1)

# Add units (assuming mg/m2, like in Max's data, but not actually specified by NILU
pts_df["variable"] = pts_df["variable"] + "_mg/m2"

# Save
pts_df.to_csv("../data/1000_lakes_nilu_dep_2017-21.csv", index=False)

pts_df.head()

## 4. Join to EMEP data

Øyvind would like the new NILU values appended to Max's data (as `2017_NILU`, `2018_NILU` etc.)

In [ ]:
par_map = {"NHy": "nh4_mg/m2", "NOx": "no3_mg/m2", "SOx": "xso4_mg/m2"}
for emep_par, nilu_par in par_map.items():
    emep_df = emep_dict[emep_par]
    nilu_df = pts_df.query("variable == @nilu_par").copy()
    assert len(nilu_df) == 1003
    del nilu_df["variable"]
    nilu_df.columns = ["StationCode"] + [f"{year}_NILU" for year in range(2017, 2022)]
    df = pd.merge(emep_df, nilu_df, how="left", on="StationCode")
    df.to_csv(
        f"../data/{emep_par}_mgpm2_1000_lakes_dep_2017-21_emep_nilu_combined.csv", index=False
    )
    df.rename({"!StationID": "StationID"}, inplace=True, axis="columns")
df